In [9]:
# Import necessary libraries
import pickle
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd



In [26]:
##load KNN

#first load function KNNWithFeatureImportance

class KNNWithFeatureImportance(KNeighborsClassifier):
    def __init__(self, n_neighbors=5, **kwargs):
        super().__init__(n_neighbors=n_neighbors, **kwargs)
        self.feature_importances_ = None
    
    def fit(self, X, y):
        # Call the original fit method
        super().fit(X, y)
        
        # Create a simple feature importance based on feature variance
        # This is just one possible approach - you could use other methods
        rf_model = RandomForestClassifier(n_estimators=50)
        rf_model.fit(X, y)
        
        self.feature_importances_ = rf_model.feature_importances_
        
        return self



# Load the pre-trained KNN model
with open(r'KNNFinal\\KNN_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load the standard used for feature scaling
with open(r'KNNFinal\\KNN_Scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

# Load the mask for selected features
with open(r'KNNFinal\\selected_features_mask.pkl', 'rb') as file:
    mask_features = pickle.load(file)

In [ ]:
##Load Naiv bayes model

# Load the pre-trained Gaussian Naive Bayes model
with open(r'BayesFinal\\naive_bayes_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load the MinMaxScaler used for feature scaling
with open(r'BayesFinal\\naive_bayes_minmax_Scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

# Load the mask for selected features
with open(r'BayesFinal\\selected_features_mask.pkl', 'rb') as file:
    mask_features = pickle.load(file)


In [ ]:
# Load the dataset and remove duplicates
data = pd.read_excel(r'..\\TestData.xlsx')
data = data.drop_duplicates()

# Separate features (X) and labels (y)
X = data.drop(columns="label")
y = data["label"]

# Encode the labels into numeric format
label_encoder = LabelEncoder()
y = pd.Series(label_encoder.fit_transform(y), index=y.index)

# Scale the features using the loaded Scaler
X_scaled = scaler.transform(X)

# Select only the features specified by the mask
X_selected = X_scaled[:, mask_features]

# Predict the labels using the loaded model
y_pred = model.predict(X_selected)
y_true = y.values

# Calculate and print the accuracy of the model
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Calculate and print the F1 score of the model
# Use 'binary' for binary classification; adjust if it's a multi-class problem
f1 = f1_score(y_true, y_pred, average='binary')
print(f"F1 Score: {f1:.4f}")

# Calculate and print the ROC AUC score of the model
roc_auc = roc_auc_score(y_true, y_pred)
print(f"ROC AUC: {roc_auc:.4f}")

Accuracy: 0.6383
F1 Score: 0.6667
ROC AUC: 0.6368
